전국초중등학교위치표준데이터.csv 에서 고등학교 명단만 따로 추출

In [22]:
# csv 파일 받아오기
import pandas as pd
df = pd.read_csv('csv/전국초중등학교위치표준데이터.csv', encoding='CP949')

df.head()

TypeError: read_csv() got an unexpected keyword argument 'columns'

In [33]:
# 고등학교만 뽑아오기

print(df[df['학교급구분'] == '고등학교']['학교명'])


5            누원고등학교
7          창원남산고등학교
8         부산서여자고등학교
10           인창고등학교
24           부석고등학교
            ...    
23908    천안여자상업고등학교
23916        간동고등학교
23925        사북고등학교
23926        장곡고등학교
23946      전주공업고등학교
Name: 학교명, Length: 4765, dtype: object
